In [4]:
data_path = "../../datas/COLD/"
datasets = ["COLD_base.json", "COLD_key.json"]

In [7]:
lexiconsDatas = ["COLD_lexicons.json"]

In [8]:
import json
from pypinyin import pinyin

chars = set()
py2char = {}
for dataset in datasets:
    with open(data_path + dataset, "r", encoding="utf-8") as f:
        datas = json.loads("".join(f.readlines()))
        for data in datas:
            for char in data["text"]:
                pys = pinyin(char, style=0, heteronym=True)[0]
                if len(pys) == 0 or pys[0] == char or char in chars:
                    continue
                chars.add(char)
                for py in pys:
                    if not py in py2char:
                        py2char[py] = set()
                    py2char[py].add(char)

for lexiconsData in lexiconsDatas:
    with open(lexiconsData, "r", encoding="utf-8") as f:
        datas = json.loads("".join(f.readlines()))
        for data in datas:
            for char in data:
                pys = pinyin(char, style=0, heteronym=True)[0]
                if len(pys) == 0 or pys[0] == char or char in chars:
                    continue
                chars.add(char)
                for py in pys:
                    if not py in py2char:
                        py2char[py] = set()
                    py2char[py].add(char)

In [9]:
import numpy as np

N = len(chars)
A = np.zeros((N, N), dtype=bool)
char2id = {}
id2char = list(chars)

for i, v in enumerate(id2char):
    char2id[v] = i

with open("char2id.json", "x", encoding="utf-8") as f:
    f.write(json.dumps(char2id, ensure_ascii=False))
with open("id2char.json", "x", encoding="utf-8") as f:
    f.write(json.dumps(id2char, ensure_ascii=False))

In [10]:
def add_jinyin(py1, py2, py, nb):
    if py1 in py:
        for char in py2char.get(py.replace(py1, py2), []):
            nb.add(char)
    if py2 in py:
        for char in py2char.get(py.replace(py2, py1), []):
            nb.add(char)
for py, chars in py2char.items():
    neighbors = set()
    for char in chars:
        neighbors.add(char)
    add_jinyin("n", "l", py, neighbors)
    add_jinyin("zh", "z", py, neighbors)
    add_jinyin("ch", "c", py, neighbors)
    add_jinyin("sh", "s", py, neighbors)
    add_jinyin("ng", "n", py, neighbors)
    
    neighbors = list(neighbors)
    for i in range(len(neighbors)-1):
        for j in range(i+1, len(neighbors)):
            id_i, id_j = char2id[neighbors[i]], char2id[neighbors[j]]
            A[id_i, id_j] = A[id_j, id_i] = 1
# 加入自环，汉字和其本身为近音关系
for i in range(N):
    A[i, i] = 1

In [11]:
np.save("homoGraphA.npy", A)

In [12]:
A.sum()

163969

In [13]:
N

3879